# Data Extraction 

In [3]:
#Imports
import pandas as pd
import numpy as np
import datetime
import os
import warnings
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

### Connect to database 

In [4]:
# Load the file 
json_data = json.loads(open('../../db_conn.json').read())

conn = 'mysql+pymysql://'+json_data["user"]+':'+json_data["password"]+'@'+json_data["host"]+':'+str(json_data["port"])+'/'+json_data["db_name"]

In [5]:
# Connect to the DB
mysql_engine = create_engine(conn,echo=False)

### Retrieve the dataset

The dataset used will have the following characteristics, which were derived from EDA step:

* We will include sales for products 45, 5, and 9 only.
* We will ignore stores related to weather stations 5 and 8.
* The model will be fitted to predict daily sales of one product across stores (in the first iteration), therefore, we will require daily sales for each of the products in every relevant store using a timestamp.

In [8]:
q = """select t.item_nbr, t.units, t.store_nbr, w.* FROM train as t
left join RetailWeather.key as s on t.store_nbr = s.store_nbr
left join RetailWeather.weather as w on s.station_nbr = w.station_nbr and t.date = w.date
where t.item_nbr in (45, 9, 5) and t.store_nbr not in (5, 8);"""

In [9]:
sales_df = pd.read_sql_query(q, conn)

In [10]:
sales_df.date = pd.to_datetime(sales_df.date)

In [11]:
sales_df.head()

,item_nbr,units,store_nbr,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,5,0,1,1,2012-01-01,52.0,31.0,42.0,NaN,36.0,...,NaT,NaT,RA FZFG BR,NaN,0.05,29.78,29.92,3.6,20.0,4.6
1,9,29,1,1,2012-01-01,52.0,31.0,42.0,NaN,36.0,...,NaT,NaT,RA FZFG BR,NaN,0.05,29.78,29.92,3.6,20.0,4.6
2,45,0,1,1,2012-01-01,52.0,31.0,42.0,NaN,36.0,...,NaT,NaT,RA FZFG BR,NaN,0.05,29.78,29.92,3.6,20.0,4.6
3,5,0,1,1,2012-01-02,50.0,31.0,41.0,NaN,26.0,...,NaT,NaT,,NaN,0.01,29.44,29.62,9.8,24.0,10.3
4,9,60,1,1,2012-01-02,50.0,31.0,41.0,NaN,26.0,...,NaT,NaT,,NaN,0.01,29.44,29.62,9.8,24.0,10.3


Now that we have what we wanted, let's start cleaning. 

# Data transformation 